In [1]:
import numpy as np
import pandas as pd
# BeautifulSoup is used to remove html tags from the text
from bs4 import BeautifulSoup 
import re # For regular expressions

# Stopwords can be useful to undersand the semantics of the sentence.
# Therefore stopwords are not removed while creating the word2vec model.
# But they will be removed  while averaging feature vectors.
from nltk.corpus import stopwords
import torch

import numpy as np
import pandas as pd
import torch
import sys
import re

from sklearn.manifold import TSNE
import plotly.graph_objects as go

import argparse
import os
from os.path import dirname, abspath
from functools import partial
import json
import yaml
import numpy as np

import torch
import torch.nn as nn
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader

import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR

from datasets import load_dataset

import seaborn as sns
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt

from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader

from datasets import Dataset
from tqdm.auto import tqdm

from sklearn.cluster import KMeans
import itertools

In [2]:
EMBED_DIMENSION = 300
EMBED_MAX_NORM = 1

data_dir = "/users/ujan/sports-language-in-politics/models/cbow/"

In [3]:
if torch.backends.mps.is_available(): device = "mps"
elif torch.cuda.is_available(): device = "cuda"
else: device = "cpu"

In [4]:
class CBOW_Model(nn.Module):
    """
    Implementation of CBOW model described in paper:
    https://arxiv.org/abs/1301.3781
    """

    def __init__(self, vocab_size: int):
        super(CBOW_Model, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=EMBED_DIMENSION,
            max_norm=EMBED_MAX_NORM,
        )
        self.linear = nn.Linear(
            in_features=EMBED_DIMENSION,
            out_features=vocab_size,
        )

    def forward(self, inputs):
        x = self.embeddings(inputs)
        x = x.mean(axis=1)
        x = self.linear(x)
        return x

In [5]:
df = pd.read_csv('/users/ujan/Downloads/Sentiment_Analysis_Dataset.csv')
print(df)

                                   id\tsentiment\treview
0      "5814_8"\t1\t"With all this stuff going down a...
1      "2381_9"\t1\t"\"The Classic War of the Worlds\...
2      "7759_3"\t0\t"The film starts with a manager (...
3      "3630_4"\t0\t"It must be assumed that those wh...
4      "9495_8"\t1\t"Superbly trashy and wondrously u...
...                                                  ...
24995  "3453_3"\t0\t"It seems like more consideration...
24996  "5064_1"\t0\t"I don't believe they made this f...
24997  "10905_3"\t0\t"Guy is a loser. Can't get girls...
24998  "10194_3"\t0\t"This 30 minute documentary Buñu...
24999  "8478_8"\t1\t"I saw this movie as a child and ...

[25000 rows x 1 columns]


In [6]:
ids = []
sents = []
revs = []
for item in df['id\tsentiment\treview']:
    id, sent, rev = item.split('\t')
    sent = int(sent)
    ids.append(id)
    sents.append(sent)
    revs.append(rev)

df = pd.DataFrame.from_dict({'id':ids, 'sentiment':sents, 'reviews':revs})

In [7]:
df.head()

,id,sentiment,reviews
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [8]:
reviews=df.iloc[:,2].values
labels=df.iloc[:,1].values

In [9]:
from bs4 import BeautifulSoup
import re

def parseHtml(html):
  soup = BeautifulSoup(html, 'html.parser')
  return soup.get_text()

def removeDigits(string):
  for i in range(10):
    string=string.replace(str(i),' ')
  return string

#removing html
reviews=list(map(parseHtml, reviews))

#removing digits
reviews=list(map(removeDigits, reviews))

/var/folders/by/rktr_w596p97pmt8_cbknvs80000gn/T/ipykernel_16098/3324077737.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


In [10]:
#tokenizing
import nltk
nltk.download('punkt')
tokenizedText=[nltk.word_tokenize(item) for item in reviews]

[nltk_data] Downloading package punkt to /users/ujan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
#removing punctuation
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
tokenizedText = [[word for word in review if word not in punc] for review in tokenizedText]

<>:2: SyntaxWarning: invalid escape sequence '\,'
<>:2: SyntaxWarning: invalid escape sequence '\,'
/var/folders/by/rktr_w596p97pmt8_cbknvs80000gn/T/ipykernel_16098/959386291.py:2: SyntaxWarning: invalid escape sequence '\,'
  punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''


In [12]:
import numpy as np
#splitting the Dataset into train and test set
totalRows=len(tokenizedText)

splitRatio=0.75
splitPoint=int(splitRatio*totalRows)

trainReviews=tokenizedText[:splitPoint]
trainLabels=labels[:splitPoint]
testReviews=tokenizedText[splitPoint:]
testLabels=labels[splitPoint:]

In [13]:
def get_embeddings_and_vocab(sub_name):
    
    vocab = torch.load(data_dir+sub_name+"_vocab.pt")
    print('vocab size: {}'.format(len(vocab.get_itos())))
    model = CBOW_Model(vocab_size=len(vocab.get_itos()))
    model = torch.load(data_dir+sub_name+"_model.pt", map_location=device)

    def normalization(embeddings):
        norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
        norms = np.reshape(norms, (len(norms), 1))
        embeddings_norm = embeddings / norms
        return embeddings_norm
    
    # embedding from first model layer
    embeddings = list(model.parameters())[0].cpu().detach().numpy()
    
    # normalization
    embeddings_norm = normalization(embeddings)

    return embeddings_norm, vocab

In [29]:
embeddings_norm, vocab = get_embeddings_and_vocab('politics_2015_9_12_100k_samples') 
#embeddings_norm, vocab = get_embeddings_and_vocab('politics_2015_10_35k_samples') 

vocab size: 9269


In [30]:
def getVectors(dataset, embeddings_norm, vocab):
    vocab_set = set(vocab.get_itos())
    singleDataItemEmbedding=np.zeros(EMBED_DIMENSION)
    vectors=[]
    for dataItem in dataset:
        wordCount=0
        for word in dataItem:
            if word in vocab_set:
                singleDataItemEmbedding = singleDataItemEmbedding + embeddings_norm[vocab[word]]
                wordCount=wordCount+1

        if wordCount > 0:
            singleDataItemEmbedding=singleDataItemEmbedding/wordCount  
        vectors.append(singleDataItemEmbedding)
    return vectors

trainReviewVectors=getVectors(trainReviews, embeddings_norm, vocab)
testReviewVectors=getVectors(testReviews, embeddings_norm, vocab)

In [31]:
from sklearn.metrics import accuracy_score
import sys
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import f1_score

def printResults(y_true, y_predicted):
  print("Accuracy= ", accuracy_score(y_true, y_predicted))

  columns=['false', 'true']
  #plot_confusion_matrix_from_data(y_true, y_predicted, columns)

  precision, recall, fscore, support = score(y_true, y_predicted)

  print('###########################################')
  print('precision: {}'.format(precision))  
  print('recall: {}'.format(recall))
  print('fscore: {}'.format(fscore))
  print('support: {}'.format(support))
  print('###########################################3')

  print('Macro F1 ',f1_score(y_true, y_predicted, average='macro'))

  print('Micro F1 ', f1_score(y_true, y_predicted, average='micro'))

In [32]:
#naive bayes
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
clfNB = MultinomialNB()

scaler = MinMaxScaler()
scaledTrainX= scaler.fit_transform(trainReviewVectors)
scaledTestX = scaler.fit_transform(testReviewVectors)
clfNB.fit(scaledTrainX, trainLabels)

#test naive bayes accuracy
testLabelsPredicted=list(clfNB.predict(scaledTestX))

#print results
print("####################RESULTS OF NAIVE BAYES CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF NAIVE BAYES CLASSIFIER##################
Accuracy=  0.56128
###########################################
precision: [0.1369201  0.97997457]
recall: [0.87090164 0.53505727]
fscore: [0.23663697 0.6921868 ]
support: [ 488 5762]
###########################################3
Macro F1  0.46441188471512307
Micro F1  0.56128


In [33]:
#neural network
from sklearn.neural_network import MLPClassifier

clfMLP = MLPClassifier(hidden_layer_sizes=(10, 10, 10))
clfMLP.fit(trainReviewVectors, trainLabels)
  
testLabelsPredicted=list(clfMLP.predict(testReviewVectors))

#print results
print("####################RESULTS OF NEURAL NETWORK CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF NEURAL NETWORK CLASSIFIER##################
Accuracy=  0.796
###########################################
precision: [0.80670103 0.78544183]
recall: [0.78766908 0.8046239 ]
fscore: [0.79707146 0.79491716]
support: [3179 3071]
###########################################3
Macro F1  0.7959943126478457
Micro F1  0.796


/Users/ujan/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [34]:
## shuffle

In [35]:
from random import shuffle
shuffle(trainReviewVectors)

In [36]:
#neural network
from sklearn.neural_network import MLPClassifier

clfMLP = MLPClassifier(hidden_layer_sizes=(10, 10, 10))
clfMLP.fit(trainReviewVectors, trainLabels)
  
testLabelsPredicted=list(clfMLP.predict(testReviewVectors))

#print results
print("####################RESULTS OF NEURAL NETWORK CLASSIFIER##################")
printResults(testLabelsPredicted, testLabels)

####################RESULTS OF NEURAL NETWORK CLASSIFIER##################
Accuracy=  0.5136
###########################################
precision: [0.59117268 0.43706294]
recall: [0.5088741  0.52004539]
fscore: [0.54694486 0.47495682]
support: [3606 2644]
###########################################3
Macro F1  0.5109508402636748
Micro F1  0.5136


/Users/ujan/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
